In [1]:
import os
import torch
import datetime
import time

import torch

import torch.nn as nn
import torch.optim as optim
from sklearn.model_selection import KFold
from torch.optim.lr_scheduler import StepLR
from torch.utils.data import DataLoader
from torch.utils.tensorboard import SummaryWriter
from tqdm import tqdm
import subprocess
import os
from utils import *
from models import *
from my_loss import *
from data_process import *

[NeMo W 2023-04-23 04:34:33 optimizers:54] Apex was not found. Using the lamb or fused_adam optimizer will error out.
[NeMo W 2023-04-23 04:34:33 nemo_logging:349] /home/ubuntu/.local/lib/python3.8/site-packages/pydub/utils.py:170: RuntimeWarning: Couldn't find ffmpeg or avconv - defaulting to ffmpeg, but may not work
      warn("Couldn't find ffmpeg or avconv - defaulting to ffmpeg, but may not work", RuntimeWarning)
    
[NeMo W 2023-04-23 04:34:34 experimental:27] Module <class 'nemo.collections.asr.modules.audio_modules.SpectrogramToMultichannelFeatures'> is experimental, not ready for production and is not fully supported. Use at your own risk.


In [2]:
CONSTANTS = InitializationTrain(
    verbose=True
)
dataset = CPCdataBinaural(metadata=CONSTANTS.metadata)

Using GPU: Tesla T4
Model Name:  None
Checking paths...
Folder already exists: /home/ubuntu/elec823/log/00_TMP_MODEL
Folder already exists: /home/ubuntu/elec823/checkpoints/00_TMP_MODEL
Folder already exists: /home/ubuntu/elec823/log/last_output/
Seed set to: 3407


In [3]:
model = EncoderPredictor().to(CONSTANTS.device)
mel = model.logmel

[NeMo I 2023-04-23 04:34:40 mixins:170] Tokenizer SentencePieceTokenizer initialized with 128 tokens


[NeMo W 2023-04-23 04:34:40 modelPT:161] If you intend to do training or fine-tuning, please call the ModelPT.setup_training_data() method and provide a valid configuration file to setup the train data loader.
    Train config : 
    manifest_filepath: /data/NeMo_ASR_SET/English/v2.0/train/tarred_audio_manifest.json
    sample_rate: 16000
    batch_size: 32
    shuffle: true
    num_workers: 8
    pin_memory: true
    use_start_end_token: false
    trim_silence: false
    max_duration: 20.0
    min_duration: 0.1
    shuffle_n: 2048
    is_tarred: true
    tarred_audio_filepaths: /data/NeMo_ASR_SET/English/v2.0/train/audio__OP_0..4095_CL_.tar
    
[NeMo W 2023-04-23 04:34:40 modelPT:168] If you intend to do validation, please call the ModelPT.setup_validation_data() or ModelPT.setup_multiple_validation_data() method and provide a valid configuration file to setup the validation data loader(s). 
    Validation config : 
    manifest_filepath:
    - /data/ASR/LibriSpeech/librispeech_withs

[NeMo I 2023-04-23 04:34:40 features:287] PADDING: 0
[NeMo I 2023-04-23 04:34:43 save_restore_connector:247] Model EncDecCTCModelBPE was successfully restored from /home/ubuntu/.cache/huggingface/hub/models--nvidia--stt_en_conformer_ctc_large/snapshots/2c8326e4e43ae5b994612cfea3f3029818fb23c6/stt_en_conformer_ctc_large.nemo.


In [4]:
train_loader = DataLoader(dataset=dataset, batch_size=3)

## Listener Info (Audiogram)

In [16]:
listener_info = ListenerInfo(['L0231', 'L0201'])
audiogram_l = [listener_info.info[i]['audiogram_l'] for i in range(len(listener_info.info))]
audiogram_r = [listener_info.info[i]['audiogram_r'] for i in range(len(listener_info.info))]
audiogram_cfs = [listener_info.info[i]['audiogram_cfs'] for i in range(len(listener_info.info))]

In [17]:
audiogram_l

[[20, 20, 20, 50, 65, 60, 75, 85], [10, 10, 20, 60, 85, 80, 85, 75]]

In [15]:
listener_info = ListenerInfo(['L0231', 'L0201'])
print(listener_info.info)
print(listener_info.info[0]['audiogram_l'])
print(listener_info.info[0]['audiogram_r'])
print(listener_info.info[0]['audiogram_cfs'])

[OrderedDict([('audiogram_l', [20, 20, 20, 50, 65, 60, 75, 85]), ('audiogram_r', [20, 15, 10, 30, 50, 50, 55, 70]), ('audiogram_cfs', [250, 500, 1000, 2000, 3000, 4000, 6000, 8000])]), OrderedDict([('audiogram_l', [10, 10, 20, 60, 85, 80, 85, 75]), ('audiogram_r', [5, 5, 10, 30, 40, 60, 85, 85]), ('audiogram_cfs', [250, 500, 1000, 2000, 3000, 4000, 6000, 8000])])]
[20, 20, 20, 50, 65, 60, 75, 85]
[20, 15, 10, 30, 50, 50, 55, 70]
[250, 500, 1000, 2000, 3000, 4000, 6000, 8000]


In [14]:
for speech_input_l, speech_input_r, info_dict in tqdm(train_loader, desc="Training:"):
    mel_feature_l, mel_feature_length = mel(
            input_signal=speech_input_l.to(device),
            length=torch.full((speech_input_l.shape[0],), speech_input_l.shape[1]).to(device),
        )
    listener_info = ListenerInfo(info_dict['listener'])
    listener_info.info['audiogram_l']
    break

Training::   0%|          | 0/1621 [00:00<?, ?it/s]


TypeError: list indices must be integers or slices, not str

In [ ]:
print(model.asr_model.cfg)

In [ ]:
for item, value in model.asr_model.cfg.items():
    print(item, value)

In [ ]:
for item, value in model.asr_model.cfg['preprocessor'].items():
    print(item, value)

In [11]:
import numpy as np
from scipy.interpolate import interp1d

# 示例数据
a = np.array([0, 1, 2, 3, 4, 5, 6, 7])  # 8元素的频率数组
b = np.array([10, 20, 30, 40, 50, 60, 70, 80])  # 8元素的值数组
c = np.linspace(0, 7, 80)  # 80元素的频率数组

# 创建线性插值函数
linear_interpolation = interp1d(a, b)

# 计算c中每个频率对应的值
result = linear_interpolation(c)

# 打印结果
print(result)


[10.         10.88607595 11.7721519  12.65822785 13.5443038  14.43037975
 15.3164557  16.20253165 17.08860759 17.97468354 18.86075949 19.74683544
 20.63291139 21.51898734 22.40506329 23.29113924 24.17721519 25.06329114
 25.94936709 26.83544304 27.72151899 28.60759494 29.49367089 30.37974684
 31.26582278 32.15189873 33.03797468 33.92405063 34.81012658 35.69620253
 36.58227848 37.46835443 38.35443038 39.24050633 40.12658228 41.01265823
 41.89873418 42.78481013 43.67088608 44.55696203 45.44303797 46.32911392
 47.21518987 48.10126582 48.98734177 49.87341772 50.75949367 51.64556962
 52.53164557 53.41772152 54.30379747 55.18987342 56.07594937 56.96202532
 57.84810127 58.73417722 59.62025316 60.50632911 61.39240506 62.27848101
 63.16455696 64.05063291 64.93670886 65.82278481 66.70886076 67.59493671
 68.48101266 69.36708861 70.25316456 71.13924051 72.02531646 72.91139241
 73.79746835 74.6835443  75.56962025 76.4556962  77.34177215 78.2278481
 79.11392405 80.        ]


In [12]:
import numpy as np

def mel_to_hz(mel):
    return 700 * (10**(mel / 2595) - 1)

def hz_to_mel(hz):
    return 2595 * np.log10(1 + hz / 700)

def get_central_frequencies(nfilt, lowfreq, highfreq):
    low_mel = hz_to_mel(lowfreq)
    high_mel = hz_to_mel(highfreq)

    mel_points = np.linspace(low_mel, high_mel, nfilt + 2)  # nfilt + 2 points to include bounds
    hz_points = mel_to_hz(mel_points)

    central_frequencies = hz_points[1:-1]  # exclude the first and last points
    return central_frequencies


nfilt = 80
lowfreq = 0
highfreq = 8000

central_frequencies = get_central_frequencies(nfilt, lowfreq, highfreq)
print(central_frequencies)


[  22.12006573   44.93912761   68.47927399   92.7632912   117.81468564
  143.6577065   170.31736924  197.81947982  226.19065969  255.4583715
  285.65094575  316.79760819  348.92850808  382.07474742  416.26841104
  451.54259764  487.93145185  525.47019727  564.19517057  604.14385667
  645.35492499  687.86826694  731.72503449  776.96768002  823.63999739
  871.78716437  921.45578634  972.69394144 1025.55122705 1080.07880785
 1136.32946531 1194.35764884 1254.21952842 1315.97304901 1379.67798665
 1445.3960063  1513.19072154 1583.12775615 1655.27480762 1729.70171269
 1806.48051495 1885.68553457 1967.39344025 2051.68332342 2138.63677481
 2228.33796343 2320.87371803 2416.33361116 2514.81004589 2616.39834519
 2721.1968443  2829.30698581 2940.83341795 3055.88409582 3174.5703859
 3297.0071739  3423.3129759  3553.6100531  3688.02453017 3826.68651734
 3969.73023633 4117.29415026 4269.52109773 4426.55843104 4588.55815882
 4755.67709321 4928.07700162 5105.92476332 5289.39253099 5478.65789735
 5673.90

In [13]:
len(central_frequencies)

80